In [1]:
import os
import pyspark

# make sure pyspark tells workers to use python3 not 2 if both are installed
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python'


conf = pyspark.SparkConf()
conf.setMaster("spark://tispark-master:7077")
sc = pyspark.SparkContext(conf=conf)

spark = pyspark.sql.SparkSession.builder \
        .master("spark://tispark-master:7077") \
        .appName("demo-tispark") \
        .config("spark.tispark.pd.addresses", "pd0:2379") \
        .getOrCreate()

In [2]:
import pytispark.pytispark as pti
 
ti = pti.TiContext(spark)
 
ti.tidbMapDatabase("mysql")
 
spark.sql("select count(*) from user").show()

+--------+
|count(1)|
+--------+
|       1|
+--------+



In [3]:
%%time

import random
num_samples = 10000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.range(0, num_samples).filter(inside).count()
pi = 4.0 * count / num_samples
print(pi)

3.1414528
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 2.13 s


In [4]:
%%time

import random
num_samples = 10000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

itr = filter(inside, range(0, num_samples))
pi = 4.0 * sum(1 for _ in itr)/num_samples
print(pi)

3.1419876
CPU times: user 3.49 s, sys: 100 ms, total: 3.59 s
Wall time: 3.65 s


In [5]:
sc.stop()